In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
#| default_exp putils

In [ ]:
#| export
import numpy as np
#import psutil
import sys, importlib, json, math, os, time, math
from datetime import datetime

# import warnings
# warnings.filterwarnings("error")

# SingletonObjects

In [ ]:
#| export
class SingletonObjects:
    "A utility for refrencing objects that should only be declared once."
    __instance = None
    @staticmethod 
    def getInstance():
        """ Static access method. """
        if SingletonObjects.__instance == None:
           SingletonObjects()
        return SingletonObjects.__instance

    def __init__(self):
        """ Virtually private constructor. """
        if SingletonObjects.__instance != None:
            raise Exception("This class is a singleton!")
        else:
            SingletonObjects.__instance = self
        self.objects = {}

    def clear(self):
        self.objects = {}
            
    def get_object(self, name=None):
        return self.objects[name]

    def add_object(self, key=None, value=None):
        if key in self.objects:
            raise Exception(f'Object {key} already exists in SingletonObjects.')            
        else:
            self.objects[key]=value


# UniqueNamer


In [ ]:
#| export
class UniqueNamer:
    "A utility for ensuring the names of functions are unique."
    __instance = None
    @staticmethod 
    def getInstance():
        """ Static access method. """
        if UniqueNamer.__instance == None:
           UniqueNamer()
        return UniqueNamer.__instance

    def __init__(self):
        """ Virtually private constructor. """
        if UniqueNamer.__instance != None:
            raise Exception("This class is a singleton!")
        else:
            UniqueNamer.__instance = self
        self.names = {}

    def clear(self, namespace=None):
        if namespace==None:
            self.names = {}
        else:
            self.names[namespace] = {}

            
    def get_name(self, namespace=None, name=None):
        # checks if name is unqiue, if a name is not unique a new one is created, name recored 
        if namespace in self.names:
            namespace_list = self.names[namespace]
        else:
            namespace_list = {}
            self.names[namespace] = namespace_list

        if name in namespace_list: 
            num = namespace_list[name]+1
            namespace_list[name]=num
            name = f'{name}{num}'
        #else:
        namespace_list[name]=0
        return name
    
    def report(self,  namespace=None, name=None,):

        if namespace is None:
            for namespace, namespace_list in self.names.items():
                print(namespace, len(namespace_list))
                for name in namespace_list:
                    print("*** ", name)
        else:
            if namespace in self.names:
                namespace_list = self.names[namespace]
                if name == None:
                    print(len(namespace_list))
                    for nname in namespace_list:
                        print("*** ", nname, namespace_list[nname])
                else:
                    print("*** ", name, namespace_list[name])

# FunctionsList

In [ ]:
#| export
class FunctionsList:
    "A utility for storing functions created, keyed on the function name."
    __instance = None
    @staticmethod 
    def getInstance():
        """ Static access method. """
        if FunctionsList.__instance == None:
             FunctionsList()
        return FunctionsList.__instance
    def __init__(self):
        """ Virtually private constructor. """
        if FunctionsList.__instance != None:
             raise Exception("This class is a singleton!")
        else:
             FunctionsList.__instance = self
        self.functions = {}

    def clear(self, namespace=None):
        if namespace==None:
            self.functions = {}
        else:
            self.functions[namespace] = {}
    
    def add_function(self, namespace=None, func=None):
        # adds a function to a namespace list, by name
        if namespace in self.functions:
            namespace_list = self.functions[namespace]
        else:
            namespace_list = {}
            self.functions[namespace]=namespace_list

        name = func.get_name()
        if name in namespace_list:
            raise Exception(f'Function {name} is already in namespace list {namespace}')
        namespace_list[name]=func
        
        return name

    def remove_function(self, namespace=None, name=None):
        if name in self.functions[namespace]:
            func = self.functions[namespace].pop(name)     
            
        if self.count(namespace)==0:
            self.functions.pop(namespace)
            
        return func
    
    def delete_function(self, namespace=None, name=None):
        func = self.remove_function(namespace=namespace, name=name)
        del func

    def get_function(self, namespace=None, name=None):     
        if  name is None:
            raise Exception(f'The function name must be specified')
        if  namespace is None:
            raise Exception(f'The namespace must be specified')
        
        if namespace in self.functions:
            namespace_list = self.functions[namespace]
        else:
            return name
        
        if isinstance(name, str) :
            if  name in namespace_list:
                func = namespace_list[name]
            else:
                raise Exception(f'Function {name} does not exist in namespace {namespace}')
        else:
            func = name

        return func
    
    def count(self, namespace=None):
        if namespace is None:
            return len(self.functions)
        
        if namespace in self.functions:
            namespace_list = self.functions[namespace]
            return len(namespace_list)
        else:
            raise Exception(f"Namespace {namespace} not found in report")
                
                
                
    def report(self, namespace=None, name=None):
        print("--- functions report")
        if namespace is None:
            for namespace, namespace_list in self.functions.items():
                print(len(namespace_list), 'NAMESPACE', namespace)
                ctr = 1
                for name, function in namespace_list.items():
                    print("*** ", ctr, name, [function])
                    print(function)
                    ctr = ctr + 1
                print()
        else:   
            if namespace in self.functions:
                namespace_list = self.functions[namespace]
            else:
                raise Exception(f"Namespace {namespace} not found in report")

            if name == None:
                print(len(namespace_list), 'NAMESPACE', namespace)
                ctr = 1
                for name, function in namespace_list.items():
                    print("*** ", ctr, name, [function])
                    print(function)
                print()
            else:
                print("*** ", name, [namespace_list[name]])
                print(namespace_list[name])          
            

# Memory

In [ ]:
#| export
class Memory:
    "A utility for recording global values."
    __instance = None
    @staticmethod 
    def getInstance():
        """ Static access method. """
        if Memory.__instance == None:
           Memory()
        return Memory.__instance

    def __init__(self):
        """ Virtually private constructor. """
        if Memory.__instance != None:
            raise Exception("This class is a singleton!")
        else:
            Memory.__instance = self
        self.memory = {}

    def clear(self):
        self.memory = {}

            
    def get_data(self, key=None):
        value = None
        if key in self.memory:
            value = self.memory[key]
        return value
    
    def add_data(self, key=None, value=None):
        self.memory[key]=value



# NumberStats

In [ ]:
#| export
class NumberStats:
    "A utility for calculating the statistice of a number."
    __instance = None
    @staticmethod 
    def getInstance():
        """ Static access method. """
        if NumberStats.__instance == None:
             NumberStats()
        return NumberStats.__instance
    def __init__(self):
        """ Virtually private constructor. """
        if NumberStats.__instance != None:
             raise Exception("This class is a singleton!")
        else:
             NumberStats.__instance = self
        self.max = -math.inf
        self.min = math.inf

    
    def add(self, number=None):
        if number > self.max:
            self.max = number

        if number < self.min:
            self.min = number
                
    def report(self):
        print("--- stats report")
        print(f'Max: {self.max:4.3f}')
        print(f'Min: {self.min:4.3f}')
            

In [ ]:
#| export
class FunctionsData():
    "Data collected for a set of functions"
    def __init__(self):
        self.data = {}
            
    def add_data(self, func):
        name = func.get_name()
        if name in self.data.keys():
            self.data[name].append(func.get_value())
        else:
            dlist=[]
            self.data[name]=dlist
            self.data[name].append(func.get_value())
    
    def add_reward(self, func):
        name = 'reward'
        if name in self.data.keys():
            self.data[name].append(func.get_reward())
        else:
            dlist=[]
            self.data[name]=dlist
            self.data[name].append(func.get_reward())
            
    def add_fitness(self, func):
        name = 'fitness'
        if name in self.data.keys():
            self.data[name].append(func.get_fitness())
        else:
            dlist=[]
            self.data[name]=dlist
            self.data[name].append(func.get_fitness())

    def add_value(self, name, value):
        if name in self.data.keys():
            self.data[name].append(value)
        else:
            dlist=[]
            self.data[name]=dlist
            self.data[name].append(value)


    def add_list(self, key, list):
        self.data[key]= list

In [ ]:
# for x in range(10.0):
#     NumberStats.getInstance().add(x)
#     NumberStats.getInstance().add(-x)

# NumberStats.getInstance().report()

In [ ]:
# from pct.functions import Proportional 

In [ ]:
# prop = Proportional()

In [ ]:
# UniqueNamer.getInstance().report()

In [ ]:
# FunctionsList.getInstance().report()

In [ ]:
#| export
def dynamic_module_import(modulename, package=None):
    if modulename not in sys.modules:
        importlib.import_module(modulename, package)     

In [ ]:
#| export
def dynamic_class_load(modulename, classname):
    module = importlib.import_module(modulename) 
    my_class = getattr(module, classname)

In [ ]:
#import importlib
#importlib.import_module('pct.functions', 'Constant')   

In [ ]:
#| export
def get_drive():
    if os.name == 'nt':
        drive = os.path.abspath(os.sep)
    else:
        drive = os.path.abspath(os.sep)+'mnt'+os.sep+'c'+os.sep    
    return drive

In [ ]:
#| export
def loadjson(file):      
    with open(file) as f:
        rtn = json.load(f)
    return rtn

In [ ]:
var = loadjson("ws.json")
print(var)

{'type': 'WeightedSum', 'name': 'weighted_sum', 'value': 35, 'links': {'0': 'constant', '1': 'constant1', '2': 'constant2'}, 'weights': [1, 1, 1]}


# Counter

In [ ]:
#| export
class Counter(object):

  def __init__(self, limit=1000, init=0, step=1, print=100, pause=False, display=10):
      self.limit=limit
      self.counter=init
      self.step=step
      self.print=print
      self.pause=pause
      self.display=display
      

  def __call__(self):
      self.counter+=self.step
      return self.counter
  
  def get(self):
      return self.counter
    
  def get_limit(self):
      return self.limit

  def set_limit(self, limit):
      self.limit=limit

# Functions

## set_dirs

In [ ]:
#| export
def set_dirs(dirs):
	if dirs is None:
		out = {}
	else:
		out = eval(dirs)

	if 'drive' not in out:
		out['drive'] = 'G:/My Drive/'
	if 'root_path' not in out:
		# out['root_path'] = 'C:/Users/ruper/'
		out['root_path'] = os.path.expanduser('~')+os.sep
	if 'configs_dir' not in out:
		out['configs_dir'] = 'Versioning/PCTSoftware/Libraries/python/pctlocal/tests/ga/pctobject/configs/'
	if 'plots_dir' not in out:
		out['plots_dir'] = '/tmp/ARC'

	# Create the directory if it doesn't exist
	if not os.path.exists(out['plots_dir']):
		os.makedirs(out['plots_dir'])
	
	return out



In [ ]:
#| export
def stringIntListToListOfInts(strList, delimiter):
    #listRes = list(strList.split(","))
    #print(listRes)
    result = []
    for item in strList.strip('][').split(','):
        result.append(int(item))
    return result

In [ ]:
#| export
def stringFloatListToListOfFloats(strList, delimiter):
    #listRes = list(strList.split(","))
    #print(listRes)
    result = []
    for item in strList.strip('][').split(','):
        result.append(float(item))
    return result

In [ ]:
#| export
def stringListToListOfStrings(strList, delimiter=','):
    #listRes = list(strList.split(","))
    #print(listRes)
    result = []
    for item in strList.strip('][').split(delimiter):
        result.append(item.strip())
    return result

In [ ]:
#| export
def listNumsToString(list):
    str = ""
    for item in list:
        str += f'{item}'
    return str

In [ ]:
#| export
def round_lists(alist, formatted, places):    
    if isinstance(alist, str):
        raise Exception(f'Value {alist} should be a number in round_lists.')

    if isinstance(alist, float) or isinstance(alist, int):
        return round(alist,places)
    
    if isinstance(alist[0], float) or isinstance(alist[0], int):
        return [round(num,places) for num in alist]
    else:
        for item in alist:    
            rtd = round_lists(item, formatted, places)
            if rtd is not None:
                formatted.append(rtd)

In [ ]:
#| export
def floatListsToString(alist, places):
    flist = []    
    if len(alist)>0:
        round_lists(alist,flist,places)
    return f'{flist}'

In [ ]:
#| export
def limit_large_float(val, limit=10000000):
    if abs(val) > limit:
        val = - np.sign(val) * limit

    return val

In [ ]:
#| export
def sigmoid(x, range, slope) :
    val = 0
    if abs(x) > 10000000:
        exv = - np.sign(x) * 10000000
    else:
        exv = -x * slope / range
    if exv > 709:
        exv = 709
    try:
        val = -range / 2 + range / (1 + np.exp(exv))
    except RuntimeWarning:
        print(f'RuntimeWarning... exv={exv} x={x} slope={slope} range={range}')

    return val

In [ ]:
#| export
def smooth(new_val, old_val, smooth_factor):
    if smooth_factor > 1 or smooth_factor < 0:
        raise Exception(f'smooth_factor {smooth_factor} should be between 0 and 1')
    val = 0
    new_val = limit_large_float(new_val)
    old_val = limit_large_float(old_val)
    try:
        val = old_val * smooth_factor + new_val * (1-smooth_factor)
    except RuntimeWarning:
        print(f'RuntimeWarning... old_val={old_val} new_val={new_val} smooth_factor={smooth_factor}')
    return val

In [ ]:
#| export
def sigmoid_array(x, range, slope) :
    exv = -x * slope / range
    return -range / 2 + range / (1 + np.exp(exv))
    

In [ ]:
#| export
def dot(inputs, weights):
    sum = 0
    for i in range(len(inputs)):
        sum += inputs[i]*weights[i]
    return sum

In [ ]:
#| export
def list_of_ones(numx):
    x = [1 for i in range(numx) ]
    return x

In [ ]:
#|gui
# li = list_of_ones(3)
# print(li)

In [ ]:
#| export
def limit_to_range(num, lower, upper):
    if num < lower:
        frac, _  = math.modf(num)
        num = abs(frac)

    if num > upper:
        frac, _ = math.modf(num)
        num = upper - frac
    return num

In [ ]:
#| export
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay                 
                 controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    
def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [ ]:
import matplotlib.pyplot as plt
x = 2 #np.linspace(-5, 5, 11)
#x = -10000001
range=2
scale=2
y = sigmoid(x, range, scale)
print(y)
# plt.plot(x, y)
# plt.grid()
# plt.xlim(-6, 6)
# plt.xlabel('x')
# plt.title('expit(x)')
# plt.show()

0.7615941559557646


In [ ]:
#| export
from pathlib import Path
import os

def is_in_notebooks():
    term = os.getenv('TERM') 
    if term == 'xterm-color':
        return True
    
    return False

In [ ]:
#| export
def printtime(msg):
    print(f'{datetime.now()} {os.getpid()} {msg}')
    return time.perf_counter()


In [ ]:
#| export
def clip_value(val, range):
    rtn = max(min(val, range[1]), range[0])
    return rtn


## get_abs_tol

In [ ]:
#| export
def get_abs_tol(key):
    # dic = {'evolve': 0.01, 'ARC-evolve' : 0.01, 'ARC-display': 0.01, 'ARC': 0.01}
    dic = { 'ARC-evolve' : 0.01, 'ARC-display': 0.1, 'ARC-change' : 0.01, 'ARC-zero': 0.01, 'ARC-gradient': 0.0001}
    dic = {}

    # if key in dic:
    return dic[key]
    
    # return 0.001

## get_rel_tol

In [ ]:
#| export
def get_rel_tol(key):
    # dic = { 'ARC-change' : 1e-3}
    dic = { }

    # if key in dic:
    return dic[key]
    
    # return 1e-6

In [ ]:
#| export
def map_to_int_odd_range(val=None, inrange=None, outrange=None):
    a = round(val)
    b = clip_value(a, inrange)
    rtn = b + (int((outrange[1] - outrange[0])/2) + 1)
    return rtn

def map_to_int_even_range(val=None, inrange=None, outrange=None):
    b = clip_value(val, inrange)
    if b == inrange[1]:
        b = b - 1
    rtn = math.floor(b) + int((outrange[1] - outrange[0] + 1 )/2) + 1
    return rtn

In [ ]:
limits = [-2, 2]
mapped = [1, 5]
vals = [-3.1, -2.1, -1.51, -1.5, -1.4, -0.9,-0.5, -0.1,  0, 0.1, 0.6, 1.1, 2.1, 210.1 ]
for val in vals:
    print(val, map_to_int_odd_range(val, limits, mapped))


-3.1 1
-2.1 1
-1.51 1
-1.5 1
-1.4 2
-0.9 2
-0.5 3
-0.1 3
0 3
0.1 3
0.6 4
1.1 4
2.1 5
210.1 5


In [ ]:

#vals = [0.6]

vals = [-3.1, -2.1, -1.51, -1.5, -1.4, -0.9, -0.5, -0.1, 0, 0.1, 0.6, 1.1, 2.1, 210.1 ]
limits = [-2, 2]
mapped = [1, 4]
for val in vals:
    print(val, map_to_int_even_range(val, limits, mapped))


-3.1 1
-2.1 1
-1.51 1
-1.5 1
-1.4 1
-0.9 2
-0.5 2
-0.1 2
0 3
0.1 3
0.6 3
1.1 4
2.1 4
210.1 4


In [ ]:
#| export 

class TimerError(Exception):
    """A custom exception used to report errors in use of Timer class"""

class Timer:
    def __init__(self):
        self._start_time = None
        self._counter = 0
        self._total_time = 0

    def start(self):
        """Start a new timer"""
        if self._start_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")

        self._start_time = time.perf_counter()
        # print(self._start_time)

    def stop(self):
        """Stop the timer, and report the elapsed time"""
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")

        elapsed_time = time.perf_counter() - self._start_time 

        self._total_time = self._total_time + elapsed_time
        self._start_time = None
        self._counter += 1
        
    def mean(self):
        mtime = self._total_time / self._counter
        self._start_time = None

        return mtime

    def total(self):
        return self._total_time
    
    def count(self):
        return self._counter 

In [ ]:
timer = Timer()
timer.start()
time.sleep(1)
timer.stop()
timer.start()
time.sleep(1)
timer.stop()

print(timer.mean())
print(timer.total())
print(timer.count())



1.0030185500000357
2.0060371000000714
2


# PCTRunProperties  

In [ ]:
#| export
class PCTRunProperties():

    @classmethod
    def get_environment_properties(cls, root=None, env=None, property_dir=None, property_file=None):

        filename=env + os.sep + property_dir + os.sep + property_file
        file = root  + os.sep + 'data' + os.sep + 'ga' + os.sep + filename

        environment_properties, env_name = PCTRunProperties.get_environment_properties_from_filename(file)

        return environment_properties, env_name

    @classmethod
    def get_environment_properties_from_filename(cls, filename):
        
        prp = PCTRunProperties()
        prp.load_db(filename)
        environment_properties = eval(prp.db['environment_properties'])
    
        return environment_properties, prp.db['env_name']


    @classmethod
    def get_properties_from_filename(cls, filename):
        
        prp = PCTRunProperties()
        prp.load_db(filename)
        props={}

        props['gym_name'] = prp.db['gym_name']
    
        return props


    def load_db(self, file):
        "Load properties from file."
        from jproperties import Properties
        skip = ['raw', 'env', 'col', '', '', '', '', '', '']
        # read properties from file
        configs = Properties()
        #print(file)
        with open(file, 'rb') as config_file:
            configs.load(config_file)

        items_view = configs.items()
        self.db = {}
        for item in items_view:
            if item[0] in skip:
                continue
            if item[0].startswith('level'):
                continue
            self.db[item[0]] = item[1].data

        if 'environment_properties' in self.db:                    
            ep = eval(self.db['environment_properties'])
            if 'reward_type' in ep:
                self.db['reward_type'] = ep['reward_type']

    def get_error_properties(self):
        "Get properties of error function from loaded properties list of the form propertyn."
        error_properties = []
        for property in range(1, 100):
            property_key = f'property{property}'
            if property_key in self.db:
                property_string = self.db[property_key]
                strarr = property_string.split(':')
                if strarr[0] == 'error':
                    parr = strarr[1].split(',')
                    prop=[]
                    prop.append(parr[0])
                    prop.append(parr[1])
                    error_properties.append(prop)

        return error_properties
    
    @classmethod
    def get_file_props(self, filepath):
        index1=filepath.rindex(os.sep)
        file = filepath[index1+1:]
        index2=filepath.rindex(os.sep, 0, index1)
        property_dir=filepath[index2+1:index1]

        index1=filepath.index(os.sep)
        index2=filepath[index1+1:].index(os.sep)

        drive=filepath[0:index2+index1+1]

        return drive, property_dir, file
        

In [ ]:
#| gui
# property_file = 'ga-000.000-s001-1x1-m007-ARC0010-9ddcf52416e60d65f19007957d07262d-consolidated.properties'
# property_dir = ''
# print(property_dir, property_file)
# ep = PCTRunProperties.get_environment_properties(root='testfiles', env='ARC', property_dir=property_dir, property_file=property_file)
# print(ep)

In [ ]:
#| export
def get_ram_mb():
    import psutil
    # Get the current process ID
    pid = os.getpid()

    # Get the memory usage of the current process
    process = psutil.Process(pid)
    memory_usage = process.memory_info().rss / 1024 ** 2

    return memory_usage

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()